In [ ]:
import sklearn
import numpy as np
import pickle
import pandas as pd

In [ ]:
with open("./df_pickle/df_pool.pkl", "rb") as fp:
    df_pool = pickle.load(fp)

In [ ]:
with open("./df_pickle/df_train.pkl", "rb") as fp:
    df_train = pickle.load(fp)

In [ ]:
with open("./df_pickle/df_test.pkl", "rb") as fp:
    df_test = pickle.load(fp)

In [ ]:
with open("./df_pickle/df_gen.pkl", "rb") as fp:
    df_gen = pickle.load(fp)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
class CommitteeMember:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Dense(128, input_dim=768*2, activation='relu'))
        self.model.add(Dense(2, activation='softmax'))

        # Compile the model
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train, epochs=25, batch_size=64)

In [ ]:
neural_net = Sequential()
neural_net.add(Dense(256, input_dim=768*2, activation='relu'))
neural_net.add(Dense(256, activation='relu'))
neural_net.add(Dense(2, activation='softmax'))

neural_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
trainset = df_pool[df_pool['label'].notna()]['combined_embedding'].values
#pool = df_pool[df_pool['label'].isna()]['combined_embedding']
labels = df_pool[df_pool['label'].notna()]['label'].values
assert len(trainset) == len(labels)

In [ ]:
df_test['combined_embedding'] = df_test.apply(lambda x: np.concatenate((x['question_embedding'],x['context_embedding'])), axis=1)
Xtest = df_test['combined_embedding'].values
ytest = df_test['label'].values

In [ ]:
df_pool.loc[df_pool[df_pool['label'].isna()]['combined_embedding'].iloc[0:1].index]

,question,context,question_embedding,context_embedding,combined_embedding,combined_embedding_str,label
3698,What are ensemble methods and how do they work?,"A subspace of a vector space RM is a line, a ...","[tensor(0.2155), tensor(0.4005), tensor(-0.070...","[tensor(-0.0080), tensor(0.1167), tensor(0.482...","[0.21554105, 0.40045273, -0.070689894, 0.01691...",[ 0.21554105 0.40045273 -0.07068989 ... 0.02...,NaN


In [21]:
#reset training set and pool
from multiprocessing import Pool
import worker
if __name__ ==  '__main__':


    committee = [worker.CommitteeMember() for _ in range(10)]
    testacc_qbc=[] #this should hold the final accuracies
    #Xtrain = trainset # should be the concatinated embeddings for question and context
    #ytrain = labels
    #Xpool = pool['combined_embedding'].values # unlabeled data in pool
    n_samples = 1000 # number of samples to be labeled from pool
    n_add = 10
    #poolidx=np.arange(len(Xpool),dtype=np.int64)
    #poolidx=np.setdiff1d(poolidx,trainset)
    for i in range(n_samples):
        Xtrain = df_pool[df_pool['label'].notna()]['combined_embedding'].values
        ytrain = df_pool[df_pool['label'].notna()]['label'].values
        Xpool = df_pool[df_pool['label'].isna()]['combined_embedding']
        ypool_lab = []

        p = Pool(processes=n_add)
        p.map(worker.task, [(i, Xtrain, ytrain, Xpool, committee_member, ypool_lab) for committee_member in committee])
        p.close()
        p.join()


        #get probability of label for each class based on voting in the committee
        ypool_p = np.mean(np.array(ypool_lab), 0)
        #Refit model in all training data
        neural_net.fit(Xtrain,ytrain)
        testacc_qbc.append((len(Xtrain),neural_net.evaluate(Xtest, ytest)))
        #select sample with maximum disagreement (least confident)
        ypool_p_sort_idx = np.argsort(-ypool_p.max(1)) #least confident
        # make labels for new points
        #ypool = ['y']*10 #None
        df_pool.loc[Xpool[ypool_p_sort_idx[-n_add:]].index]['label'] = 'y'
        #add to training set
        #Xtrain = np.concatenate((Xtrain,Xpool[ypool_p_sort_idx[-n_add:]]))
        #ytrain = np.concatenate((ytrain,ypool[ypool_p_sort_idx[-n_add:]]))
        #remove from pool
        #Xpool = np.delete(Xpool, ypool_p_sort_idx[-n_add:])
        #poolidx=np.setdiff1d(poolidx,poolidx[ypool_p_sort_idx[-n_add:]])
        print('Model: LR, %i samples (QBC)'%(n_samples+i*n_add))

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-26 14:12:43         1420
metadata.json                                  2023-03-26 14:12:43           64
variables.h5                                   2023-03-26 14:12:43       797864
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-26 14:12:44         1420
metadata.json                                 

ValueError: setting an array element with a sequence.